In [20]:
#Import dependencies
import pandas as pd

In [21]:
#Read PPMA sas table
PPMA = pd.read_sas('..\..\SAS\ppma.sas7bdat')

In [22]:
#Read POLO sas table
POLO = pd.read_sas('..\..\SAS\polo.sas7bdat')

In [23]:
#Remove duplicate party ids in ordered table to get most recent update
POLO.drop_duplicates(subset = 'PARTY_ID', inplace=True, keep='first')
PPMA.drop_duplicates(subset = 'PARTY_ID', inplace=True, keep='first')

In [37]:
#SAS table is dumb do not recommend this method
def clean_sas(df):
    dict_list = []
    TOTAL=df.shape[0]
    count=0
    counts = [100,90,80,70,60,50,40,30,20,10,0]
    for row in df.itertuples():
        count+=1
        percent = int(count/TOTAL*100)
        if percent in counts:
            counts.pop()
            print(f'{percent}% DONE!')
        new_dict = {}
        new_dict['PARTY_ID'] = int(row.PARTY_ID)
        new_dict['POST_CD_ID'] = int(row.POST_CD_ID)
        new_dict['FROM_DT'] = row.FROM_DT
        try:
            new_dict['ADDR_1'] = (str(row.ADDR_1).split("'"))[1]
        except:
            new_dict['ADDR_1'] = 'N/A'
        try:
            new_dict['ADDR_2'] = (str(row.ADDR_2).split("'"))[1]
        except:
            new_dict['ADDR_2'] = 'N/A'
        try:
            new_dict['ADDR_3'] = (str(row.ADDR_3).split("'"))[1]
        except:
            new_dict['ADDR_3'] = 'N/A'
        try:
            new_dict['CITY'] = (str(row.CITY).split("'"))[1]
        except:
            new_dict['CITY'] = 'N/A'
        new_dict['STATE_CD'] = (str(row.STATE_CD).split("'"))[1]
        try:
            new_dict['ZIP'] = (str(row.ZIP).split("'"))[1]
        except:
            new_dict['ZIP'] = 'N/A'
        dict_list.append(new_dict)
    new_df = pd.DataFrame(dict_list)
    return(new_df)

In [59]:
#clean sas tables
POLO_DF = clean_sas(POLO)

0% DONE!
10% DONE!
20% DONE!
30% DONE!
40% DONE!
50% DONE!
60% DONE!
70% DONE!
80% DONE!
90% DONE!
100% DONE!


In [38]:
#clean sas tables
PPMA_DF=clean_sas(PPMA)

0% DONE!
10% DONE!
20% DONE!
30% DONE!
40% DONE!
50% DONE!
60% DONE!
70% DONE!
80% DONE!
90% DONE!
nan
nan
nan
100% DONE!


In [40]:
#read insurance table
insurance = pd.read_csv('address_updates_2019_dates.csv')
insurance.head()

,PARTY_ID,ME,POST_CD_ID,ADDR_TYPE,ADDR_1,ADDR_2,ADDR_3,CITY,STATE_CD,ZIP,FROM_DT
0,3570268,49522010277,13139660,Billing,3857 Cooper St,NaN,NaN,Jackson,MI,49201,8/23/2019
1,1962871,1803980566,13139660,Billing,3857 Cooper St,NaN,NaN,Jackson,MI,49201,8/23/2019
2,4817046,2507110595,13155193,Billing,11900 E 12 Mile Rd,Ste 110,NaN,Warren,MI,48093,8/21/2019
3,3508042,1642070653,15594580,Billing,920 Country Club Rd,Ste 220B,Lane Surgery Center,Eugene,OR,97401,8/21/2019
4,3514872,1720071793,16586724,Billing,601 W 2nd St,NaN,NaN,Bloomington,IN,47403,8/21/2019


In [60]:
#Merge polo and insurance
polo_insurance = pd.merge(POLO_DF, insurance, on='PARTY_ID', suffixes= ('_masterfile','_insurance'))
polo_insurance.head()

,ADDR_1_masterfile,ADDR_2_masterfile,ADDR_3_masterfile,CITY_masterfile,FROM_DT_masterfile,PARTY_ID,POST_CD_ID_masterfile,STATE_CD_masterfile,ZIP_masterfile,ME,POST_CD_ID_insurance,ADDR_TYPE,ADDR_1_insurance,ADDR_2_insurance,ADDR_3_insurance,CITY_insurance,STATE_CD_insurance,ZIP_insurance,FROM_DT_insurance
0,Carr 188,Loiza Home Of The Elderly,N/A,Loiza,2019-08-26 21:08:28.993166924,5170070,30085906,PR,00772,4204120264,29908561,Billing,Carr 188 Int 187,"Concilio De Salud Integral De Loiza, Inc",NaN,Loiza,PR,772,6/22/2019
1,1522 11th Ave S,Camellia Medical Group,N/A,Birmingham,2019-08-26 21:08:28.991859913,2694250,30086019,AL,35205,102020251,27668659,Billing,3686 Grandview Pkwy,Physician's Plaza Suite 810,NaN,Birmingham,AL,35243,5/1/2019
2,12214 Lakewood Blvd,Ste 110,N/A,Downey,2019-08-26 21:08:28.971540928,2713491,30085373,CA,90242,49522740542,29244345,Billing,12214 Lakewood Blvd,NaN,NaN,Downey,CA,90242,4/6/2019
3,7816 E Commercial St,N/A,N/A,Broken Arrow,2019-08-26 21:08:26.355173111,2584273,30085974,OK,74014,4707810829,30042604,Billing,7816 E Commercial St,74014,NaN,Broken Arrow,OK,74014,8/7/2019
4,3800 S National Ave,Ste LL105,N/A,Springfield,2019-08-26 21:08:26.342437983,2319690,30085939,MO,65807,2803780440,30039940,Billing,3800 S National Ave Ste LL105,NaN,NaN,Springfield,MO,65807,8/6/2019


In [56]:
#Merge ppma and insurance
ppma_insurance = pd.merge(PPMA_DF, insurance, on='PARTY_ID', suffixes= ('_masterfile','_insurance'))
ppma_insurance.head()

,ADDR_1_masterfile,ADDR_2_masterfile,ADDR_3_masterfile,CITY_masterfile,FROM_DT_masterfile,PARTY_ID,POST_CD_ID_masterfile,STATE_CD_masterfile,ZIP_masterfile,ME,POST_CD_ID_insurance,ADDR_TYPE,ADDR_1_insurance,ADDR_2_insurance,ADDR_3_insurance,CITY_insurance,STATE_CD_insurance,ZIP_insurance,FROM_DT_insurance
0,455 W 200 N,Unit 213,N/A,Salt Lake Cty,2019-08-26 21:08:32.633399963,5940983,30085879,UT,84103,87501120365,29815634,Billing,30 N 1900 E Rm 4A100,NaN,NaN,Salt Lake Cty,UT,84132,8/22/2019
1,40503 Manor Drive,N/A,N/A,Magnolia,2019-08-26 21:08:32.627401114,5170717,30085486,TX,77354,4802122212,28641136,Billing,2801 Devine St Ste 101,NaN,NaN,Columbia,SC,29205,7/19/2019
2,1350 Boylston St,Unit 303,N/A,Boston,2019-08-26 21:08:32.604870081,5711227,30085752,MA,02215,49632130031,27563902,Billing,6560 Fannin St,"Scurlock Tower, Suite 802",NaN,Houston,TX,77030,5/1/2019
3,3693 Via Monte Napoleone Dr,N/A,N/A,Kissimmee,2019-08-26 21:08:32.600316048,2230738,30085633,FL,34759,3845890861,26831039,Billing,339 Cypress Pkwy,Ste 180,NaN,Kissimmee,FL,34759,12/22/2018
4,100 W Water St,Apt 316,N/A,Elmira,2019-08-26 21:08:32.593358040,9614324,30085431,NY,14901,5002190810,28904651,Billing,600 Roe Ave,NaN,NaN,Elmira,NY,14905,7/26/2019


In [51]:
#Define summarizing function
def sum_it_up(df):
    count =0 
    add_count = 0
    zip_count = 0
    state_count = 0
    cd_count  =0 
    dict_list = []
    date_count = 0
    city_count = 0
    for row in df.itertuples():
        count += 1
        if row.POST_CD_ID_masterfile != row.POST_CD_ID_insurance:
            cd_count +=1
            new_dict = {}
            new_dict['ME'] = row.ME
            new_dict['ADDR_1_insurance'] = row.ADDR_1_insurance
            new_dict['ADDR_1_masterfile'] = row.ADDR_1_masterfile
            new_dict['ADDR_2_insurance'] = row.ADDR_2_insurance
            new_dict['ADDR_2_masterfile'] = row.ADDR_2_masterfile
            new_dict['ADDR_3_insurance'] = row.ADDR_3_insurance
            new_dict['ADDR_3_masterfile'] = row.ADDR_3_masterfile
            new_dict['ZIP_insurance'] = row.ZIP_insurance
            new_dict['ZIP_masterfile'] = row.ZIP_masterfile
            new_dict['CITY_insurance'] = row.CITY_insurance
            new_dict['CITY_masterfile'] = row.CITY_masterfile
            new_dict['DATE_insurance'] = row.FROM_DT_insurance
            new_dict['DATE_masterfile'] = row.FROM_DT_masterfile
            new_dict['STATE_insurance'] = row.STATE_CD_insurance
            new_dict['STATE_masterfile'] = row.STATE_CD_masterfile
            dict_list.append(new_dict)
#             if row.FROM_DT_insurance > row.FROM_DT_masterfile:
#                 date_count += 1
            if row.ADDR_1_insurance != row.ADDR_1_masterfile:
                add_count +=1
            if row.CITY_insurance != row.CITY_masterfile:
                city_count +=1
            if row.ZIP_insurance != row.ZIP_masterfile:
                zip_count +=1
            if row.STATE_CD_insurance != row.STATE_CD_masterfile:
                state_count +=1
    print(f'''
    {cd_count/count * 100}% ({cd_count}) POST_CD_IDs do not match 
    {add_count/cd_count * 100}% ({add_count}) addresses do not match
    {zip_count/cd_count * 100}% ({zip_count})zipcodes do not match
    {state_count/cd_count * 100}% ({state_count})states do not match
    {city_count/cd_count * 100}% ({city_count})cities do not match
    ''')
    return(dict_list)

In [65]:
#Summarize polo differences
polo_list = sum_it_up(polo_insurance)
POLO_DF_differences = pd.DataFrame(polo_list)


    77.36483385065043% (311812) POST_CD_IDs do not match 
    84.21356458378767% (262588) addresses do not match
    66.93841160699395% (208722)zipcodes do not match
    25.845381191230615% (80589)states do not match
    70.62524854720151% (220218)cities do not match
    


In [63]:
#Summarize ppma differences
ppma_list = sum_it_up(ppma_insurance)
PPMA_DF_differences = pd.DataFrame(ppma_list)


    77.83878761256314% (343768) POST_CD_IDs do not match 
    87.35600753996881% (300302) addresses do not match
    71.23874240767029% (244896)zipcodes do not match
    24.650636475762724% (84741)states do not match
    73.4227734984059% (252404)cities do not match
    


In [66]:
#Write Results to csv
PPMA_DF_differences.to_csv('PPMA_DF.csv')
POLO_DF_differences.to_csv('POLO_DF.csv')